# Test, Trace and Preemptively Isolate

In [31]:
from epyc import JSONLabNotebook, ParallelLab
import epyc
import epydemic
import numpy as np
from epydemic import SEIR, SIR, ERNetwork, Monitor, ProcessSequence, rng, SynchronousDynamics
import matplotlib.pyplot as plt
from parameters import *

In [32]:
create_data_output_directory()

In [33]:
# The code below was taken from a GitHub repo by Simon Dobson at https://github.com/simoninireland/introduction-to-epidemics/blob/master/src/seir.ipynb (last accessed 2023-11-09) and modified to implement a preemptive quarantine strategy (see symptoms()).
class AdaptiveSEIR(SEIR):
    P_DETECT = 'pDetect'  #: Parameter for the probability  of detecting an exposed neighbour of an infected node.
    P_REWIRE = 'pRewire'  #: Parameter for the probability of rewiring an SE or SI edge.

    def __init__(self):
        super(AdaptiveSEIR, self).__init__()
        self._pDetect = None
        self._pRewire = None

    def build(self, params):
        params[self.P_SYMPTOMS] = params[self.P_REMOVE]
        params[self.P_REMOVE] = params[self.P_REMOVE] 

        super(AdaptiveSEIR, self).build(params)

        # store the parameters for later
        self._pDetect = params[self.P_DETECT]
        self._pRewire = params[self.P_REWIRE]

        self.trackNodesInCompartment(epydemic.SEIR.SUSCEPTIBLE)
        self.trackNodesInCompartment(epydemic.SEIR.REMOVED)


    def quarantine(self, n):
        g = self.network()
        rng = np.random.default_rng()
        ms = list(g.neighbors(n))
        for m in ms:
            if self.getCompartment(m) == self.SUSCEPTIBLE and rng.random() <= self._pRewire:
                # a susceptible neighbour, remove link to us
                self.removeEdge(n, m)

                # rewire to another random susceptible
                mprime = self.locus(self.SUSCEPTIBLE).draw()
                self.addEdge(m, mprime)

    def symptoms(self, t, n):
        # perform a normal becoming-symptomatic event
        super(AdaptiveSEIR, self).symptoms(t, n)

        g = self.network()
        rng = np.random.default_rng()

        # examine all neighbours and look for exposed AND susceptible nodes to quarantine
        ms = list(g.neighbors(n))
        for m in ms:
            m_compartment = self.getCompartment(m)
            if (m_compartment == self.EXPOSED or m_compartment == self.SUSCEPTIBLE) and rng.random() <= self._pDetect: # Note: this line has been modified to implement a preemptive quarantine strategy. 
                self.quarantine(m)

        # quarantine the newly symptomatic node
        self.quarantine(n)
    # END Copied Code
    def atEquilibrium(self, t):
        return len(self.compartment(self.INFECTED)) == 0 and len(self.compartment(self.EXPOSED)) == 0

    

### Create epyc experiments and run simulations

In [34]:
def ex_1_seir(lab):
    # set the disease parameter space
    lab[AdaptiveSEIR.P_EXPOSED] = p_exposed
    lab[AdaptiveSEIR.P_INFECT_ASYMPTOMATIC] = p_infect
    lab[AdaptiveSEIR.P_INFECT_SYMPTOMATIC] = p_infect
    lab[AdaptiveSEIR.P_REMOVE] = p_remove
        
    lab[AdaptiveSEIR.P_REWIRE] = p_rewire
    lab[AdaptiveSEIR.P_DETECT] = p_detect

    lab['ens'] = range(ens)

    # set the topology for the generated network
    lab[ERNetwork.N] = n
    lab[ERNetwork.KMEAN] = k_mean

    # create the model, network generator, and experiment
    p = AdaptiveSEIR()
    g = ERNetwork()
    e = epydemic.StochasticDynamics(p, g)

    # run the experiment
    lab.runExperiment(e)

In [ ]:
# Partition the controlled variables.
pds = np.split(p_detect, 100)

In [35]:
lab = ParallelLab(JSONLabNotebook(get_out_path("ex_1_seir"), create=True), cores)

In [36]:
for i, pd in enumerate(pds):
    p_detect = pd
    lab.createWith("ex_" + str(i), ex_1_seir)

/home/hf39/Documents/CS5199/.venv/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


KeyboardInterrupt: 